In [5]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend,layers
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
import matplotlib.pyplot as plt
%matplotlib inline
from pathlib import Path

In [10]:
def load_row_dataset(path):
    p = Path(path)
    row_data = pd.read_excel(p)
    row_data = row_data.iloc[:,[2,3,26,33,34,36,39,40,49,50,51,52,53,54,60,61,65]]
    return row_data

def missing_value(row_data):
    count = 0
    for j in range(0,15):
        for i in range(0,len(row_data)-1):
            if(np.isnan(np.array(row_data.iloc[[i],[j]]))):
                count+=1
                row_data.iloc[[i],[j]] = float(np.array(row_data.iloc[[i-1],[j]]))
    return row_data

def normalize(train):
    train_norm = train.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)))
    return train_norm


#WATCH!!!!!!!!!!!!!!!!!!!!!!!
def buildTrain(train, pastDay, futureDay):
    X_train, Y_train = [], []
    for i in range(train.shape[0]-futureDay-pastDay):
        X_train.append(np.array(train.iloc[i:i+pastDay]))
        Y_train.append(np.array(train.iloc[i+pastDay:i+pastDay+futureDay][" - MARKET VALUE"]))
    return np.array(X_train), np.array(Y_train)


def shuffle(X,Y,seed):
    np.random.seed(seed)
    randomList = np.arange(X.shape[0])
    np.random.shuffle(randomList)
    return X[randomList], Y[randomList]

def splitData(X,Y,rate):
    X_train = X[int(X.shape[0]*rate):]
    Y_train = Y[int(Y.shape[0]*rate):]
    X_val = X[:int(X.shape[0]*rate)]
    Y_val = Y[:int(Y.shape[0]*rate)]
    return X_train, Y_train, X_val, Y_val

#watch!!!!!!!!!!!!!!!!!!!!!!!!!
def buildManyToManyModel(shape):
    model = tf.keras.Sequential()
    model.add(layers.LSTM(17, input_length=shape[1], input_dim=shape[2]))
  # output shape: (5, 1)
    model.add(layers.Dense(1))
    model.compile(loss="mse", optimizer="adam")
    model.summary()
    return model



In [26]:
row_data = load_row_dataset("A(塑化).xlsx")
#load dataset
row_data = missing_value(row_data)
#dealing missing value
train_norm = normalize(row_data)
#normalize data
X_train, Y_train = buildTrain(train_norm, 10, 1)
#build training data set (pastday,futureday)
X_train, Y_train = shuffle(X_train, Y_train,10)
#shuffle traing data set (X,Y,seed)
X_train, Y_train, X_val, Y_val = splitData(X_train, Y_train, 0.7)
#create validation data set (X,Y,ratio)

model = buildManyToManyModel(X_train.shape)
callback = [tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10, verbose=1, mode="auto")]

model.fit(X_train, Y_train, epochs=1000, batch_size=128,callbacks=callback)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 17)                2380      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 18        
Total params: 2,398
Trainable params: 2,398
Non-trainable params: 0
_________________________________________________________________
Train on 521 samples
Epoch 1/1000
521/521 [==============================] - 3s 6ms/sample - loss: 0.0701
Epoch 2/1000
521/521 [==============================] - 0s 217us/sample - loss: 0.0439
Epoch 3/1000
521/521 [==============================] - 0s 241us/sample - loss: 0.0267
Epoch 4/1000
521/521 [==============================] - 0s 386us/sample - loss: 0.0155
Epoch 5/1000
521/521 [==============================] - 0s 334us/sample - loss: 0.0103
Epoch 6/1000
521/521 [==============================] - 0s 2

Epoch 87/1000
521/521 [==============================] - 0s 228us/sample - loss: 0.0011
Epoch 88/1000
521/521 [==============================] - 0s 213us/sample - loss: 0.0011
Epoch 89/1000
521/521 [==============================] - 0s 211us/sample - loss: 0.0011
Epoch 90/1000
521/521 [==============================] - 0s 211us/sample - loss: 0.0011
Epoch 91/1000
521/521 [==============================] - 0s 210us/sample - loss: 0.0011
Epoch 92/1000
521/521 [==============================] - 0s 185us/sample - loss: 0.0011
Epoch 93/1000
521/521 [==============================] - 0s 187us/sample - loss: 0.0011
Epoch 94/1000
521/521 [==============================] - 0s 188us/sample - loss: 0.0011
Epoch 95/1000
521/521 [==============================] - 0s 197us/sample - loss: 0.0011
Epoch 96/1000
521/521 [==============================] - 0s 197us/sample - loss: 0.0011
Epoch 97/1000
521/521 [==============================] - 0s 192us/sample - loss: 0.0011
Epoch 98/1000
521/521 [=========

In [27]:
result = model.predict(X_val)

In [28]:
from ipywidgets import interact_manual
def show(number):
    print("answer:",Y_val[number])
    print("predict:",result[number])
interact_manual(show,number = (0,len(result)-1))

interactive(children=(IntSlider(value=607, description='number', max=1214), Button(description='Run Interact',…

<function __main__.show(number)>